In [17]:
import secrets
import time
import math

In [2]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

In [3]:
def RMPT(p, s):
    q = p - 1
    k = 0 
    
    # if newP % 2 have remainder then stop
    while q % 2 != 1: 
        q //= 2 
        k += 1 

    for i in range(s):
        #Generate random a 
        a = secrets.randbelow(p-2) + 2 

        while gcd(a,p) != 1:
            a = secrets.randbelow(p-2) + 2
            
        z = pow(a, q, p) # z = a^q mod p

        # checking a^q != 1 mod p and a^q != -1 mod p 
        if z != 1 and z != p-1:
            for j in range(1, k):
                z = pow(z, 2, p)
                if z == 1:
                    return False

            if z != p-1:
                return False
    return True

In [4]:
# Check the result 
print(RMPT(58536828032471131348456426324506659436967195846753702250333704608966791528534067798278728747981548972696886762937373961684727824495513261968904996453134494716095278694362125703914706608360435446792042241177272588181082115012537677828367957226772929218486970585153142173891310402614698182537457744036891034121, 2))

False


In [5]:
def generate_prime(bits, rounds):
    while(True):
        num = secrets.randbits(bits) # using CSRPNG to generate

        # if number is even make it odd 
        if num % 2 == 0:
            num += 1

        elif num % 3 == 0:
            continue 

        elif num % 5 == 0:
            continue

        elif num % 7 == 0:
            continue

        elif num % 11 == 0: 
            continue

        elif RMPT(num, rounds):
            return num


In [61]:
def split_message(message, n):
    n_max = n-1
    blocks = []
    str_msg = str(message)

    i = 0
    while i < len(str_msg):
        block = str_msg[i:i+len(str(n_max))]
        block_int = int(block)
        if block_int >= n:
            block = str_msg[i:i+len(str(n_max))-1]
        blocks.append(int(block))
        i += len(str(n_max))

    return blocks


In [62]:
n = 1591
message = 28562810173
blocks = split_message(message, n)
print(blocks)

[285, 281, 173]


In [46]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_blocks = split_message(ascii_message, N)
    
    for i in encrypted_blocks:
        encrypted_message += pow(i, e, N)
    
    return encrypted_message

In [28]:
def RSA_decrypt(encrypted_message, d, p, q):
    dp = d % (p-1)
    dq = d % (q-1)
    qinv = pow(q, p-2, p) 

    m1 = pow(encrypted_message, dp, p)
    m2 = pow(encrypted_message, dq, q)

    h =  (qinv * (m1 - m2)) % p 
    message = m2 + h * q 

    padding_len = message[-1]
    plain_text = message[:-padding_len]

    return plain_text

In [29]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [30]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

In [31]:
def RSA_generate_d(e, p, q):
    N = p * q
    phiN = (p-1)*(q-1)
    assert gcd(e, phiN) == 1, "GCD of e and Phi N must be 1"
    d = pow(e, -1, phiN)

    return d


## RSA 2048 

User of RSA encrypt message using RSA 1024

Generate primes p and q 

User deliberately generate a weak d exponent by setting max bit length to 1/3 * N^1/4 to get faster decryption 

Encryption exponent e (2048 bit) is obtained by inversing weak d 

In [32]:
phi_test = False
e = 65537

while phi_test == False: 
    p = generate_prime(1024, 3)
    q = generate_prime(1024, 3)
    
    phiN = (p-1) * (q-1)
    N = p * q

    if gcd(e, phiN) == 1:
        phi_test = True
    

    d = RSA_generate_d(e, p, q)
    print('E is: ', e)
    print('E bit length: ', e.bit_length())

    print('D is: ', d)
    print('d bit length: ', d.bit_length())


E is:  65537
E bit length:  17
D is:  3137059610361319908709445904443926951680352381178175837742160730619120425363545301191977228230266428497821796950827411070722731959644580578448700996857816988493636239991596322538773052177981860531751034260582700729693786526425156251573860403010141225209618204319880496231973098938745667638699651479866518999078756953697471587389498472245796774986444086222765049468961140172579424404436681760592569371080777731638175926605330766221011211701969671442136829365564543008015516818152126230413990264471645031962855419728456510575398759167499398580107598292031252142031686263394948822322554154842048366638383626618553976041
d bit length:  2045


In [47]:
message = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed"

ascii_message = convert_to_ascii(message)
ascii_message.bit_length()

encrypted_message = RSA_encrypt(ascii_message, e, N)
print('Encrypted message in ascii: ', encrypted_message)
print('Encrypted message in String: ', convert_to_string(encrypted_message))

TypeError: can only concatenate str (not "int") to str

## Decryption using Chinese Remainder Theorem

In [22]:

start_time = time.time()
decrypted_message = RSA_decrypt(encrypted_message, d, p, q)
end_time = time.time()

elapsed_time = end_time - start_time

print('Elapsed time: %.10f seconds' % elapsed_time)
print('Decrypted message in ascii: ', decrypted_message)
print('Decrypted message in string: ', convert_to_string(decrypted_message))


Elapsed time: 0.0249681473 seconds
Decrypted message in ascii:  76111114101109032105112115117109032100111108111114032115105116032097109101116044032099111110115101099116101116117114032097100105112105115099105110103032101108105116044032115101100
Decrypted message in string:  Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed
